# Understanding Runtime Context and Session Management in AgentCore Runtime

## Overview

In this tutorial, we will learn how to understand and work with runtime context and session management in Amazon Bedrock AgentCore Runtime. This example demonstrates how AgentCore Runtime handles sessions, maintains context across multiple invocations, and how agents can access runtime information through the context object.

Amazon Bedrock AgentCore Runtime provides isolated sessions for each user interaction, enabling agents to maintain context and state across multiple invocations while ensuring complete security isolation between different users.

### Tutorial Details

|Information| Details|
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Context and Session Management|
| Agent type          | Single         |
| Agentic Framework   | Strands Agents |
| LLM model           | Anthropic Claude Haiku 4.5 |
| Tutorial components | Runtime Context, Session Management, AgentCore Runtime, Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Intermediate                                                                     |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3|

### Tutorial Architecture

In this tutorial, we will explore how Amazon Bedrock AgentCore Runtime manages sessions and provides context to agents. We'll demonstrate:

1. **Session Continuity**: How the same session ID maintains context across multiple invocations
2. **Context Object**: How agents can access runtime information through the context parameter
3. **Session Isolation**: How different session IDs create completely isolated environments
4. **Payload Flexibility**: How to pass custom data to agents through the payload

For demonstration purposes, we will use a Strands Agent that showcases these session management capabilities.

    
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### Tutorial Key Features

* **Session-based Context Management**: Understanding how AgentCore Runtime maintains context within sessions
* **Runtime Session Lifecycle**: Learning about session creation, maintenance, and termination
* **Context Object Access**: Accessing runtime information like session ID through the context parameter
* **Session Isolation**: Demonstrating how different sessions provide complete isolation
* **Payload Handling**: Flexible data passing through custom payload structures
* **Cross-invocation State**: Maintaining agent state across multiple calls within the same session

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

## Understanding Amazon Bedrock AgentCore Runtime Sessions

Before diving into the code, it's important to understand how Amazon Bedrock AgentCore Runtime manages sessions:

### Session Isolation and Security

AgentCore Runtime provides **complete session isolation** through dedicated microVMs:

- **Dedicated Resources**: Each session runs in its own microVM with isolated CPU, memory, and filesystem
- **Security Boundaries**: Complete separation between user sessions prevents data contamination
- **Deterministic Cleanup**: After session completion, the microVM is terminated and memory is sanitized

### Session Lifecycle

Sessions in AgentCore Runtime follow a specific lifecycle:

1. **Creation**: A new session is created on first invocation with a unique `runtimeSessionId`
2. **Active State**: Session processes requests and maintains context
3. **Idle State**: Session waits for next invocation while preserving context
4. **Termination**: Session ends due to:
   - Inactivity (15 minutes)
   - Maximum lifetime (8 hours)
   - Health check failures

### Context Persistence

Within a session, AgentCore Runtime maintains:
- **Conversation History**: Previous interactions and responses
- **Application State**: Variables and objects created during execution
- **File System**: Any files created or modified during the session
- **Environment Variables**: Custom settings and configurations

### Session Management Best Practices

- **Unique Session IDs**: Generate unique session IDs for each user or conversation
- **Context Reuse**: Use the same session ID for related invocations to maintain context
- **Session Boundaries**: Use different session IDs for different users or unrelated conversations
- **Ephemeral Nature**: Don't rely on sessions for permanent data storage (use AgentCore Memory for persistence)

In [1]:
!pip install --force-reinstall -U -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.31.6 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
sagemaker-studio 1.1.1 requires pydynamodb>=0.7.4, which is not installed.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.42.7 which is incompatible.
autogluon-multimodal 1.4.0 requires jsonschema<4.24,>=4.18, but you have jsonschema 4.25.1 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.57.1 which is incompatible.
auto

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agent to AgentCore Runtime to demonstrate session management and context handling. Our agent will showcase how to:

1. **Access Runtime Context**: Use the `context` parameter to get session information
2. **Handle Custom Payloads**: Process structured data passed through the payload
3. **Maintain Session State**: Keep track of user interactions within a session
4. **Demonstrate Session Boundaries**: Show how different sessions are isolated

### Understanding the Context Object

The `context` object in AgentCore Runtime provides valuable information about the current execution environment:

- **session_id**: The current runtime session identifier
- **Runtime Metadata**: Information about the runtime environment
- **Execution Details**: Context about the current invocation

### Strands Agent with Context Handling

Let's look at our implementation that demonstrates session management and context handling:

In [2]:
%%writefile strands_claude_context.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

@tool
def get_time():
    """ Get current time """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""
    You're a helpful assistant. You can do simple math calculations, 
    tell the weather, and provide the current time.
    Always start by acknowledging the user's name 
    """
)

def get_user_name(user_id):
    users = {
        "1": "Arun",
        "2": "Vishwa",
        "3": "Rohit",
        "4": "Rafael",
        "5": "Dhawal"
    }
    return users[user_id]
    
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    """
    AgentCore Runtime entrypoint that demonstrates context handling and session management.
    
    Args:
        payload: The input payload containing user data and request information
        context: The runtime context object containing session and execution information
    
    Returns:
        str: The agent's response incorporating context information
    """
    user_input = payload.get("prompt")
    user_id = payload.get("user_id")
    user_name = get_user_name(user_id)
    
    # Access runtime context information
    print("=== Runtime Context Information ===")
    print("User id:", user_id)
    print("User Name:", user_name)
    print("User input:", user_input)
    print("Runtime Session ID:", context.session_id)
    print("Context Object Type:", type(context))
    print("=== End Context Information ===")
    
    # Create a personalized prompt that includes context information
    prompt = f"""My name is {user_name}. Here is my request: {user_input}
    
    Additional context: This is session {context.session_id}. 
    Please acknowledge my name and provide assistance."""
    
    response = agent(prompt)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Writing strands_claude_context.py


## Understanding Session Management in AgentCore Runtime

The code above demonstrates several key concepts about how AgentCore Runtime manages sessions and provides context to agents:

### Context Object Structure

The `context` parameter in your entrypoint function provides access to runtime information:

```python
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    # Access session information
    session_id = context.session_id
    # Use context information in your agent logic
```

### Session Continuity Benefits

Within a single session, AgentCore Runtime provides:

1. **Persistent Environment**: Variables and state persist across invocations
2. **Context Preservation**: The agent can reference previous interactions
3. **Resource Reuse**: Initialized models and tools remain loaded
4. **Performance Benefits**: Reduced cold start times for subsequent invocations

### Session Isolation Guarantees

AgentCore Runtime ensures complete isolation between sessions:

- **Security**: Each session runs in its own microVM with isolated resources
- **Privacy**: No data leakage between different user sessions
- **Reliability**: Issues in one session don't affect others
- **Cleanup**: Complete memory sanitization after session termination

### Payload Flexibility

The `payload` parameter allows flexible data passing:

```python
# Example payload structures
payload = {
    "prompt": "User's question",
    "user_id": "1",
    "preferences": {...},
    "context_data": {...}
}
```

This enables rich, structured communication between clients and agents while maintaining the session context provided by the runtime.

### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [3]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_context.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_context"
)

Entrypoint parsed: file=/home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/11-AgentCore-runtime/03-advanced-concepts/02-understanding-runtime-context/strands_claude_context.py, bedrock_agentcore_name=strands_claude_context
Configuring BedrockAgentCore agent: strands_claude_context


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/11-AgentCore-runtime/03-advanced-concepts/02-understanding-runtime-context/Dockerfile
Generated .dockerignore: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/11-AgentCore-runtime/03-advanced-concepts/02-understanding-runtime-context/.dockerignore
Setting 'strands_claude_context' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/11-AgentCore-runtime/03-advanced-concepts/02-understanding-runtime-context/.bedrock_agentcore.yaml


In [5]:
cat Dockerfile

FROM ghcr.io/astral-sh/uv:python3.12-bookworm-slim
WORKDIR /app

# Configure UV for container environment
ENV UV_SYSTEM_PYTHON=1 UV_COMPILE_BYTECODE=1



COPY requirements.txt requirements.txt
# Install from requirements file
RUN uv pip install -r requirements.txt




RUN uv pip install aws-opentelemetry-distro>=0.10.1


# Set AWS region environment variable

ENV AWS_REGION=us-west-2
ENV AWS_DEFAULT_REGION=us-west-2


# Signal that this is running in Docker for host binding logic
ENV DOCKER_CONTAINER=1

# Create non-root user
RUN useradd -m -u 1000 bedrock_agentcore
USER bedrock_agentcore

EXPOSE 8080
EXPOSE 8000

# Copy entire project (respecting .dockerignore)
COPY . .

# Use the full module path

CMD ["opentelemetry-instrument", "python", "-m", "strands_claude_context"]


### Launching the context-aware agent to AgentCore Runtime

Now that we've got a docker file, let's launch our context-aware agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

Our agent will demonstrate how AgentCore Runtime manages sessions and provides context information to agents.

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [6]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_context' to account 455933813601 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_context
✅ ECR repository available: 455933813601.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_context
Getting or creating execution role for agent: strands_claude_context
Using AWS region: us-west-2, account ID: 455933813601
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-969e6d3fbf


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-strands_claude_context


Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-969e6d3fbf
Starting execution role creation process for agent: strands_claude_context
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-west-2-969e6d3fbf
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-969e6d3fbf
✓ Role created: arn:aws:iam::455933813601:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-969e6d3fbf
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-strands_claude_context
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::455933813601:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-969e6d3fbf
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_claude_context
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-969e6d3fbf
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-969e6d3fbf
Creati

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [7]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude_context


'READY'

## Demonstrating Session Management and Context Handling

Now let's demonstrate the key session management features of AgentCore Runtime by testing different scenarios:

### Scenario 1: Session Continuity
We'll use the same session ID for multiple invocations to show how context is maintained.

### Scenario 2: Session Isolation
We'll use different session IDs to demonstrate complete isolation between sessions.

### Scenario 3: Context Information Access
We'll show how agents can access runtime context information.

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [9]:
import uuid
import json
from IPython.display import Markdown, display

# Create a session ID for demonstrating session continuity
session_id = uuid.uuid4()
print(f"📋 Starting Session 1: {session_id}")
print(f"👤 User: Arun (ID: 1)")
print(f"❓ First question about weather\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How is the weather outside?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

📋 Starting Session 1: ce3a821c-63e1-4492-9cf5-b15f46f64334
👤 User: Arun (ID: 1)
❓ First question about weather



Great news, Arun! The weather outside is **sunny**. It looks like a beautiful day out there! ☀️

Is there anything else I can help you with?

In [10]:
# Continue with the same session ID to demonstrate session continuity
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Arun (ID: 1)")
print(f"❓ Follow-up question about math\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How much is 2X5?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

🔄 Continuing Session 1: ce3a821c-63e1-4492-9cf5-b15f46f64334
👤 Same user: Arun (ID: 1)
❓ Follow-up question about math



The answer is **10**, Arun! 

2 × 5 = 10

Is there anything else you'd like me to calculate for you?

In [11]:
# Continue with the same session ID - notice how the agent remembers the previous calculation
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Arun (ID: 1)")
print(f"❓ Building on previous answer - demonstrates context continuity\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "and that plus 34?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

🔄 Continuing Session 1: ce3a821c-63e1-4492-9cf5-b15f46f64334
👤 Same user: Arun (ID: 1)
❓ Building on previous answer - demonstrates context continuity



The answer is **44**, Arun!

10 + 34 = 44

So to recap: 2 × 5 = 10, and 10 + 34 = 44. Is there anything else you'd like me to help you with?

In [12]:
# NEW SESSION - Demonstrate session isolation
# Create a completely new session ID to show that context is lost
new_session_id = uuid.uuid4()
print(f"🆕 Starting NEW Session 2: {new_session_id}")
print(f"👤 Same user: Arun (ID: 1)")
print(f"❓ Attempting to reference previous calculation - should fail due to session isolation\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "And plus 10?",
    "user_id": "1"
}, session_id=str(new_session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

🆕 Starting NEW Session 2: 796c3ad2-cade-4bd0-9ca5-13240724945f
👤 Same user: Arun (ID: 1)
❓ Attempting to reference previous calculation - should fail due to session isolation



Hello Arun! It's great to meet you! 👋

I'd be happy to help you with your math calculation. However, I notice your request says "And plus 10?" - it seems like you might be asking me to add 10 to something, but I'm not sure what number you'd like me to add 10 to.

Could you please clarify what you'd like me to calculate? For example:
- Do you have a specific number you want to add 10 to?
- Or did you mean something else?

Once you provide the complete calculation, I'll solve it for you right away! 😊

In [14]:
# NEW SESSION AND USER - Demonstrate complete isolation
different_user_session = uuid.uuid4()
print(f"🆕 Starting Session 3: {different_user_session}")
print(f"👤 Different user: Vishwa (ID: 2)")
print(f"❓ Same question as first user - demonstrates user isolation\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How is the weather?",
    "user_id": "2"
}, session_id=str(different_user_session))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

🆕 Starting Session 3: 8d358030-83e1-455a-86b8-6486754d1cd6
👤 Different user: Vishwa (ID: 2)
❓ Same question as first user - demonstrates user isolation



Great news, Vishwa! The weather is currently **sunny**. It's a beautiful day out there! ☀️

Is there anything else I can help you with, such as calculations, the current time, or other information?

## Understanding the Session Management Results

The demonstrations above showcase several key aspects of AgentCore Runtime's session management:

### 1. Session Continuity (Session 1)
- **First invocation**: Agent responds to weather question and acknowledges user name
- **Second invocation**: Agent performs calculation (2×5=10)
- **Third invocation**: Agent references previous result ("that plus 34" = 44)

**Key Learning**: The agent maintained context across multiple invocations within the same session, remembering the calculation result from the previous interaction.

### 2. Session Isolation (Session 2)
- **New session ID**: Created a completely new session
- **Same user**: Used the same user ID but different session
- **Context loss**: Agent cannot reference previous calculation

**Key Learning**: Even with the same user, a new session creates a completely isolated environment with no access to previous context.

### 3. User and Session Isolation (Session 3)
- **Different user**: Mani instead of Maira
- **New session**: Complete isolation from previous sessions
- **Fresh context**: Agent starts with clean state

**Key Learning**: Each session provides complete isolation, ensuring privacy and security between different users and interactions.

### 4. Context Object Usage
Throughout all invocations, the agent:
- Accessed the runtime context via `context.session_id`
- Processed custom payload data (`user_id`, `prompt`)
- Maintained logging and debugging information

**Key Learning**: The context object provides valuable runtime information that agents can use for enhanced functionality and debugging.

### Session Management Best Practices Demonstrated

1. **Use consistent session IDs** for conversational continuity
2. **Generate unique session IDs** for different users or conversations
3. **Leverage context information** for enhanced agent behavior
4. **Design for session boundaries** - don't assume persistence across sessions
5. **Handle graceful context loss** when sessions change or expire

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [16]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

('455933813601.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_context',
 'strands_claude_context-GFjgvg6YWu',
 'bedrock-agentcore-strands_claude_context')

In [ ]:
import boto3

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!

You have successfully implemented and tested session management and context handling with Amazon Bedrock AgentCore Runtime! 

## What you've learned:

### Session Management Fundamentals
* **Session Continuity**: How the same session ID maintains context across multiple invocations
* **Session Isolation**: How different session IDs create completely isolated environments
* **Context Preservation**: How agents can maintain state and reference previous interactions
* **Security Boundaries**: How AgentCore Runtime ensures complete isolation between users

### Runtime Context Handling
* **Context Object Access**: How to access runtime information via the `context` parameter
* **Session Information**: How to retrieve and use session IDs in your agent logic
* **Payload Processing**: How to handle structured data passed through custom payloads
* **Runtime Metadata**: How agents can access execution environment information

### AgentCore Runtime Architecture
* **MicroVM Isolation**: Each session runs in its own isolated microVM
* **Resource Management**: Dedicated CPU, memory, and filesystem per session
* **Security Model**: Complete memory sanitization after session termination
* **Lifecycle Management**: Session states (active, idle, terminated) and timeouts

### Best Practices Implementation
* **Session ID Generation**: Creating unique identifiers for different conversations
* **Context Utilization**: Leveraging runtime context for enhanced agent behavior
* **State Management**: Understanding ephemeral vs persistent state
* **Error Handling**: Graceful handling of context loss and session boundaries